In [42]:
import pandas as pd
import numpy as np
from scipy.stats import zscore, mstats

# Let's try to replicate the MSCI Momentum Index methodology

- Uses both 6-month and 12-month returns
- Final score winsorized at +/-3
- Final weight capped at 7%

- Exclude some tickers that would duplicate A vs B shares:

In [43]:
# Define a list of tickers to exclude
tickers_to_exclude = ['GOOG-USA', 'NWS-USA', 'FOX-USA']

In [44]:
# Load market data
market_data = pd.read_csv('market_data.csv')

# Convert date column to datetime
market_data['date'] = pd.to_datetime(market_data['date'])

# Filter out the specified tickers
market_data = market_data[~market_data['Ticker'].isin(tickers_to_exclude)]

# Extract year and month from date
market_data['year_month'] = market_data['date'].dt.to_period('M')

# Sort data by Ticker and date
market_data = market_data.sort_values(by=['Ticker', 'date'])

# Calculate monthly closing prices
monthly_prices = market_data.groupby(['Ticker', 'year_month']).agg({'Price': 'last'}).reset_index()

# Create a pivot table with Tickers as rows and year_month as columns
pivot_prices = monthly_prices.pivot(index='Ticker', columns='year_month', values='Price')

In [45]:
pivot_prices

year_month,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07
Ticker,,,,,,,,,,,,,
A-USA,121.77,121.07,111.82,103.37,127.80,139.03,130.10,137.36,145.51,137.04,130.41,129.63,139.49
AAL-USA,16.75,14.73,12.81,11.15,12.43,13.74,14.23,15.68,15.35,13.51,11.50,11.33,10.76
AAPL-USA,196.45,187.87,171.21,170.77,189.95,192.53,184.40,180.75,171.48,170.33,192.25,210.62,218.80
ABBV-USA,149.58,146.96,149.06,141.18,142.39,154.97,164.40,176.05,182.10,162.64,161.24,171.52,186.78
ABNB-USA,152.19,131.55,137.21,118.29,126.34,136.14,144.14,157.47,164.96,158.57,144.93,151.63,138.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL-USA,112.75,103.54,91.03,93.54,105.13,114.36,112.44,127.05,129.24,130.70,141.02,135.63,133.48
YUM-USA,137.67,129.38,124.94,120.86,125.55,130.66,129.49,138.42,138.65,141.25,137.43,132.46,132.17
ZBH-USA,138.15,119.12,112.22,104.41,116.31,121.70,125.60,124.36,131.98,120.28,115.15,108.53,111.37


In [46]:
# Calculate 6-month price change excluding the most recent month
momentum_value_6m = (pivot_prices.iloc[:, -2] / pivot_prices.iloc[:, -7]) - 1

# Create DataFrame for 6-month momentum
momentum_df_6m = pd.DataFrame(momentum_value_6m, columns=['Momentum Value 6M'])

# Calculate 12-month price change excluding the most recent month
momentum_value_12m = (pivot_prices.iloc[:, -2] / pivot_prices.iloc[:, -13]) - 1

# Calculate the 12-month price change excluding the most recent month (already calculated as momentum_value)
momentum_df_12m = pd.DataFrame(momentum_value_12m, columns=['Momentum Value 12M'])

# Merge 6-month and 12-month momentum DataFrames
momentum_df = momentum_df_6m.join(momentum_df_12m)

# Filter out rows where either 6M or 12M momentum is zero
momentum_df = momentum_df[(momentum_df['Momentum Value 6M'] != 0) & (momentum_df['Momentum Value 12M'] != 0)]

# Calculate weighted average momentum
momentum_df['Weighted Momentum'] = (momentum_df['Momentum Value 6M'] + momentum_df['Momentum Value 12M']) / 2

# Merge market cap data
latest_market_data = market_data.sort_values('date').drop_duplicates('Ticker', keep='last')
momentum_df = momentum_df.merge(latest_market_data[['Ticker', 'Market Cap (millions)']], left_index=True, right_on='Ticker')

In [47]:
# Drop rows where required columns have NaN values
momentum_df.dropna(subset=['Momentum Value 6M', 'Momentum Value 12M', 'Market Cap (millions)'], inplace=True)

# Standardize the 'Weighted Momentum' with z-scores
momentum_df['Final Z-score'] = zscore(momentum_df['Weighted Momentum'])

# Winsorize the final Z-scores (limit extreme values)
momentum_df['Clipped Z-score'] = momentum_df['Final Z-score'].clip(-3, 3)

# Calculate the Momentum Score as per MSCI methodology
def compute_momentum_score(z):
    if z > 0:
        return 1 + z
    else:
        return (1 - z) ** -1

momentum_df['Momentum Score'] = momentum_df['Clipped Z-score'].apply(compute_momentum_score)

# Sort and limit to top 125 by Momentum Score
momentum_df = momentum_df.sort_values(by="Momentum Score", ascending=False).head(125)

# Remove '-USA' suffix from tickers in the momentum_df
momentum_df['Ticker'] = momentum_df['Ticker'].str.replace('-USA', '')


In [48]:
# Calculate weights based on Clipped Z-score and Market Cap
momentum_df['Weight_unnormalized'] = momentum_df['Clipped Z-score'] * momentum_df['Market Cap (millions)']
total_weight = momentum_df['Weight_unnormalized'].sum()
momentum_df['Final Weight'] = (momentum_df['Weight_unnormalized'] / total_weight) * 100

In [49]:
momentum_df = momentum_df.sort_values(by="Final Weight",ascending=False)
momentum_df

,Momentum Value 6M,Momentum Value 12M,Weighted Momentum,Ticker,Market Cap (millions),Final Z-score,Clipped Z-score,Momentum Score,Weight_unnormalized,Final Weight
91175,1.007899,1.643754,1.325827,NVDA,2.551758e+06,5.865347,3.000000,4.000000,7.655274e+06,25.992224
8645,0.245168,0.445616,0.345392,AMZN,1.890988e+06,1.283375,1.283375,2.283375,2.426847e+06,8.239960
85673,0.124176,0.330525,0.227351,MSFT,3.143271e+06,0.731720,0.731720,1.731720,2.299994e+06,7.809251
74931,0.402364,0.991816,0.697090,LLY,7.510199e+05,2.927003,2.927003,3.927003,2.198237e+06,7.463754
80695,0.292408,0.582611,0.437510,META,1.015056e+06,1.713879,1.713879,2.713879,1.739683e+06,5.906807
...,...,...,...,...,...,...,...,...,...,...
64975,0.204298,0.196617,0.200457,IP,1.595122e+04,0.606037,0.606037,1.606037,9.667023e+03,0.032823
91961,0.118912,0.391019,0.254966,NWSA,1.028310e+04,0.860776,0.860776,1.860776,8.851449e+03,0.030054
53447,0.063884,0.284319,0.174101,GEN,1.613578e+04,0.482864,0.482864,1.482864,7.791395e+03,0.026454
84887,0.254705,0.091359,0.173032,MRO,1.558994e+04,0.477865,0.477865,1.477865,7.449894e+03,0.025295


### Add caps to portfolio

MSCI caps the weight, though it's not totally clear what the methodology is to do that. They also have rules during reconstitution.
Let's cap the weight at 7%, which is around NVDA weight in SPMO, then ensure the rest of the portfolio is weighted accordingly.

In [50]:
# Sample data preparation (assuming previous calculations and DataFrame `momentum_df`)
# For illustration, making sure the column names match your data.
# Ensure that the DataFrame is loaded or created with appropriate columns
momentum_df['Weight_unnormalized'] = momentum_df['Market Cap (millions)'] * momentum_df['Momentum Score']
total_weight = momentum_df['Weight_unnormalized'].sum()
momentum_df['Final Weight'] = (momentum_df['Weight_unnormalized'] / total_weight) * 100  # Normalize to percentages

# Cap weights at 7%
momentum_df['Capped Weight'] = momentum_df['Final Weight'].clip(upper=7)

# Calculate the excess weight from capping
excess_weight = momentum_df['Final Weight'].sum() - momentum_df['Capped Weight'].sum()

# Handle cases where weights should be redistributed
iteration = 0
while excess_weight > 0.01 and iteration < 10:  # Using tolerance of 0.01 and max iterations to prevent infinite loops
    iteration += 1
    # Compute weights of securities below 7%
    not_capped = momentum_df['Capped Weight'] < 7.0
    
    # Total weight of securities that are not capped but still need redistribution
    not_capped_total_weight = momentum_df.loc[not_capped, 'Capped Weight'].sum()
    
    # If no weights are to be redistributed, break the loop
    if not_capped_total_weight == 0:
        break
    
    # Calculate the amount to redistribute
    redistribution = excess_weight / not_capped_total_weight
    
    # Apply redistribution, ensuring no weight exceeds the cap
    momentum_df.loc[not_capped, 'Capped Weight'] += (momentum_df.loc[not_capped, 'Capped Weight'] * redistribution).clip(upper=(7 - momentum_df.loc[not_capped, 'Capped Weight']))

    # Recalculate the excess weight
    excess_weight = 100 - momentum_df['Capped Weight'].sum()

# Assign the adjusted final weights
momentum_df['Adjusted Final Weight'] = momentum_df['Capped Weight']

# Remove '-USA' suffix from tickers in the momentum_df
momentum_df['Ticker'] = momentum_df['Ticker'].str.replace('-USA', '')

In [51]:
momentum_df = momentum_df.round(decimals=2)
momentum_df

,Momentum Value 6M,Momentum Value 12M,Weighted Momentum,Ticker,Market Cap (millions),Final Z-score,Clipped Z-score,Momentum Score,Weight_unnormalized,Final Weight,Capped Weight,Adjusted Final Weight
91175,1.01,1.64,1.33,NVDA,2551758.00,5.87,3.00,4.00,10207032.00,20.12,7.00,7.00
8645,0.25,0.45,0.35,AMZN,1890988.19,1.28,1.28,2.28,4317835.12,8.51,7.00,7.00
85673,0.12,0.33,0.23,MSFT,3143270.73,0.73,0.73,1.73,5443264.49,10.73,7.00,7.00
74931,0.40,0.99,0.70,LLY,751019.85,2.93,2.93,3.93,2949257.22,5.81,7.00,7.00
80695,0.29,0.58,0.44,META,1015056.01,1.71,1.71,2.71,2754738.72,5.43,7.00,7.00
...,...,...,...,...,...,...,...,...,...,...,...,...
64975,0.20,0.20,0.20,IP,15951.22,0.61,0.61,1.61,25618.24,0.05,0.07,0.07
91961,0.12,0.39,0.25,NWSA,10283.10,0.86,0.86,1.86,19134.55,0.04,0.05,0.05
53447,0.06,0.28,0.17,GEN,16135.78,0.48,0.48,1.48,23927.17,0.05,0.06,0.06
84887,0.25,0.09,0.17,MRO,15589.94,0.48,0.48,1.48,23039.83,0.05,0.06,0.06


In [52]:
momentum_df.to_csv("momentum_MSCI_proxy.csv")

### How closely does this match the MSCI portfolio?

In [53]:
# Load CSV data
momentum_proxy_df = pd.read_csv("momentum_MSCI_proxy.csv")
mtum_holdings_df = pd.read_csv("MTUM_holdings.csv")

# Strip trailing spaces from the 'Ticker' column in mtum_holdings_df
mtum_holdings_df['Ticker'] = mtum_holdings_df['Ticker'].str.strip()

# Part 1: Finding common tickers
momentum_tickers = set(momentum_proxy_df['Ticker'])
mtum_tickers = set(mtum_holdings_df['Ticker'])

common_tickers = momentum_tickers.intersection(mtum_tickers)
num_common_tickers = len(common_tickers)

print(f"Number of common tickers: {num_common_tickers}")
print(f"Common tickers: {common_tickers}")

Number of common tickers: 69
Common tickers: {'DECK', 'AVGO', 'NTAP', 'MU', 'DPZ', 'URI', 'ISRG', 'AMAT', 'JPM', 'PWR', 'VST', 'SMCI', 'AMZN', 'DVA', 'C', 'APH', 'LDOS', 'LLY', 'WDC', 'PGR', 'CRWD', 'TDG', 'BSX', 'GDDY', 'GM', 'WFC', 'CTAS', 'ETN', 'AMD', 'CPRT', 'HLT', 'GRMN', 'IR', 'NXPI', 'AXP', 'CMG', 'NRG', 'PANW', 'META', 'TT', 'RSG', 'PH', 'CEG', 'GD', 'GE', 'HIG', 'CAT', 'PHM', 'LRCX', 'QCOM', 'INTU', 'UBER', 'STX', 'WM', 'MCK', 'MPC', 'FICO', 'WAB', 'NFLX', 'RCL', 'COST', 'KKR', 'NVDA', 'AXON', 'KLAC', 'ALL', 'SYF', 'ANET', 'HWM'}


In [54]:
# Extract and rename columns
momentum_proxy_df = momentum_proxy_df[['Ticker', 'Adjusted Final Weight']]
momentum_proxy_df = momentum_proxy_df.rename(columns={'Adjusted Final Weight': 'Proxy Weight'})

mtum_holdings_df = mtum_holdings_df[['Ticker', 'Weight (%)']]
mtum_holdings_df = mtum_holdings_df.rename(columns={'Weight (%)': 'MTUM Weight'})

# Merge the dataframes on 'Ticker', using outer join to include all tickers
combined_df = pd.merge(momentum_proxy_df, mtum_holdings_df, on='Ticker', how='outer')

# Fill NaN values with 0 for weights
combined_df['Proxy Weight'].fillna(0, inplace=True)
combined_df['MTUM Weight'].fillna(0, inplace=True)

/tmp/ipykernel_12835/2991764532.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Proxy Weight'].fillna(0, inplace=True)
/tmp/ipykernel_12835/2991764532.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

In [55]:
combined_df.sort_values(by="Proxy Weight", ascending=False).head(20)

,Ticker,Proxy Weight,MTUM Weight
108,META,7.00,4.31
11,AMZN,7.00,4.52
122,NVDA,7.00,6.73
104,LLY,7.00,5.28
113,MSFT,7.00,0.00
17,AVGO,5.80,6.30
79,GOOGL,5.80,0.00
97,JPM,2.72,4.72
181,WMT,2.65,0.00
37,COST,2.23,3.14
